In [ ]:
# !pip install qiskit

# VQLS with Qiskit
**Course**: Applied Quantum Algorithms

**Date**: April 20, 2022

In [13]:
from qiskit import QuantumCircuit
from qiskit.circuit.library.n_local import TwoLocal
from qiskit.opflow import X, Y, Z, I, H

In [14]:
A = 0.55 * I ^ I ^ I \
    + 0.225 * I ^ Z ^ I \
        + 0.225 * I ^ I ^ Z

B = H ^ H ^ H

In [24]:
ansatz = TwoLocal(3, 'ry', 'cz', 'full', reps=2, insert_barriers=True)
# qc = QuantumCircuit(3)
# qc += ansatz
print(ansatz.decompose().draw())

     ┌──────────┐ ░           ░ ┌──────────┐ ░           ░ ┌──────────┐
q_0: ┤ Ry(θ[0]) ├─░──■──■─────░─┤ Ry(θ[3]) ├─░──■──■─────░─┤ Ry(θ[6]) ├
     ├──────────┤ ░  │  │     ░ ├──────────┤ ░  │  │     ░ ├──────────┤
q_1: ┤ Ry(θ[1]) ├─░──■──┼──■──░─┤ Ry(θ[4]) ├─░──■──┼──■──░─┤ Ry(θ[7]) ├
     ├──────────┤ ░     │  │  ░ ├──────────┤ ░     │  │  ░ ├──────────┤
q_2: ┤ Ry(θ[2]) ├─░─────■──■──░─┤ Ry(θ[5]) ├─░─────■──■──░─┤ Ry(θ[8]) ├
     └──────────┘ ░           ░ └──────────┘ ░           ░ └──────────┘
